In [632]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [633]:
# Load and examine basic structure
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 81 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       1200 non-null   int64  
 1   PropertyClass            1200 non-null   int64  
 2   ZoningCategory           1200 non-null   object 
 3   RoadAccessLength         977 non-null    float64
 4   LandArea                 1200 non-null   int64  
 5   RoadType                 1200 non-null   object 
 6   ServiceLaneType          75 non-null     object 
 7   PlotShape                1200 non-null   object 
 8   LandElevation            1200 non-null   object 
 9   UtilityAccess            1200 non-null   object 
 10  PlotConfiguration        1200 non-null   object 
 11  LandSlope                1200 non-null   object 
 12  District                 1200 non-null   object 
 13  NearbyTransport1         1200 non-null   object 
 14  NearbyTransport2        

In [634]:
train_df.shape
train_df.info()
null_value_percentages=(train_df.isna().sum()/train_df.shape[0])*100
null_value_percentages
rows_to_drop=null_value_percentages[null_value_percentages<5].sort_values(ascending=False)
rows_to_drop
rows_to_drop=rows_to_drop.keys()
rows_to_drop
for row in rows_to_drop:
	print("row: ",row,"\tmissing values: \t",train_df[row].isna().sum())
train_df.shape
for row in rows_to_drop:
    if(null_value_percentages[row]<6):
        train_df.drop(labels=train_df.index[train_df[row].isna()],inplace=True)

train_df.shape
columns_to_drop=null_value_percentages[null_value_percentages>40]
columns_to_drop=columns_to_drop.keys()

print("Columns to drop")
print(columns_to_drop)
train_df.drop(columns=columns_to_drop,inplace=True)
test_df.drop(columns=columns_to_drop,inplace=True)

train_df.shape
train_df.drop_duplicates(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 81 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       1200 non-null   int64  
 1   PropertyClass            1200 non-null   int64  
 2   ZoningCategory           1200 non-null   object 
 3   RoadAccessLength         977 non-null    float64
 4   LandArea                 1200 non-null   int64  
 5   RoadType                 1200 non-null   object 
 6   ServiceLaneType          75 non-null     object 
 7   PlotShape                1200 non-null   object 
 8   LandElevation            1200 non-null   object 
 9   UtilityAccess            1200 non-null   object 
 10  PlotConfiguration        1200 non-null   object 
 11  LandSlope                1200 non-null   object 
 12  District                 1200 non-null   object 
 13  NearbyTransport1         1200 non-null   object 
 14  NearbyTransport2        

In [635]:
# train_df.replace({
#     "ParkingQuality": {np.nan: "NoParking"},
#     "ParkingCondition": {np.nan: "NoParking"}
# }, inplace=True)

In [636]:
# test_df.replace({
#     "ParkingQuality": {np.nan: "NoParking"},
#     "ParkingCondition": {np.nan: "NoParking"}
# }, inplace=True)

In [637]:
import matplotlib.pyplot as plt
# plt.hist(train_df['HotelValue'], bins=50)
percentile_threshold = train_df['HotelValue'].quantile(0.999)
print("Original maximum value:", train_df['HotelValue'].max())
# Cap the values at the 99.9th percentile
train_df['HotelValue'] = train_df['HotelValue'].clip(upper=percentile_threshold)
print("Maximum value after capping:", train_df['HotelValue'].max())

Original maximum value: 745000.0
Maximum value after capping: 618227.2130000071


In [638]:
missing_summary = (
    train_df.isnull().sum() / len(train_df) * 100
).sort_values(ascending=False).to_frame("missing_percent")

# Add dtypes for reference
missing_summary["dtype"] = train_df.dtypes

# Focus only on numeric columns with some missingness
numeric_missing = missing_summary[
    (missing_summary["missing_percent"] > 0) &
    ((missing_summary["dtype"]=="float64") |
    (missing_summary["dtype"]=="int64"))
]

print(numeric_missing)

'''There are three features with missing values for numeric values.
The first is RoadAccessLength which is the length of road access available to the property. It could be that some properties do not have road
access at all, in which case we should fill 0 in all these columns, however we see that the column "RoadType" has no null values, which means 
every property has some road leading up to it. So the values are missing randomly, and not due to there being no road available. The missing
percentage is not very high (18.58%) so we can impute the values using median.'''

road_access_median = train_df["RoadAccessLength"].median()
train_df["RoadAccessLength"] = train_df["RoadAccessLength"].fillna(road_access_median)
test_df["RoadAccessLength"] = test_df["RoadAccessLength"].fillna(road_access_median)

'''FacadeArea only missing for rows with FacadeType null, we can impute 0 here as they probably have no facade.'''
train_df["FacadeArea"] = train_df["FacadeArea"].fillna(0)
test_df["FacadeArea"] = test_df["FacadeArea"].fillna(0)

'''ParkingConstructionYear imputed with median with ConstructionYear as there seems to be high correlation'''
train_df['ParkingConstructionYear'] = train_df['ParkingConstructionYear'].fillna(train_df['ConstructionYear'])
test_df['ParkingConstructionYear'] = test_df['ParkingConstructionYear'].fillna(test_df['ConstructionYear'])


                         missing_percent    dtype
RoadAccessLength               18.588640  float64
ParkingConstructionYear         4.991394  float64


In [639]:
# import math
# numeric_cols = train_df.select_dtypes(include=[np.number]).columns
# numeric_cols = numeric_cols.drop('HotelValue')

# nunique_threshold = 15
# discrete_numeric_cols = []
# for col in numeric_cols:
#     if train_df[col].nunique() < nunique_threshold and col != 'HotelValue':
#         discrete_numeric_cols.append(col)

# print(f"Found {len(discrete_numeric_cols)} discrete numeric columns:")
# continuous_numeric_cols = [
#     col for col in numeric_cols 
#     if train_df[col].nunique() >= nunique_threshold
# ]

# # Dynamically determine the grid size for the subplots
# n_cols = len(continuous_numeric_cols)
# n_rows = math.ceil(n_cols / 3) # Creates a grid with 3 columns

# fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5 * n_rows))
# axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

# for idx, feature in enumerate(continuous_numeric_cols):
#  axes[idx].scatter(train_df[feature], train_df['HotelValue'], alpha=0.5)
#  axes[idx].set_xlabel(feature)
#  axes[idx].set_ylabel('HotelValue')
#  axes[idx].set_title(f'{feature} vs HotelValue')

# # Hide any unused subplots
# for i in range(n_cols, len(axes)):
#  fig.delaxes(axes[i])

# plt.tight_layout()
# plt.show()

In [640]:
# '''Adding binary indicators for whether certain features exist for a house or not as there are a lot of facilities like terrace, parking
# etc. which do not exist for quite a few houses.'''
# # List all features with this pattern
# area_features = ['EnclosedVerandaArea', 'SeasonalPorchArea', 'ScreenPorchArea', 
#                  'TerraceArea', 'BasementTotalSF', 
#                  'ParkingArea', 'OpenVerandaArea', 'FacadeArea', 'BasementFacilitySF1', 'BasementFacilitySF2', 
#                  'BasementUnfinishedSF', 'UpperFloorArea', 'LowQualityArea', 'SwimmingPoolArea', 'ExtraFacilityValue']

# for feature in area_features:
#     train_df[f'Has{feature.replace("Area", "").replace("SF", "")}'] = (train_df[feature] > 0).astype(int)
#     test_df[f'Has{feature.replace("Area", "").replace("SF", "")}'] = (test_df[feature] > 0).astype(int)

#SwimmingPoolArea and ExtraFacilityValue show hardly any correlation to the target variable even for houses that do have those facilities.
#Thus, these can be dropped.

train_df = train_df.drop(columns=['LowQualityArea', 'SeasonalPorchArea', 'ScreenPorchArea', 'SwimmingPoolArea', 'ExtraFacilityValue'])
test_df = test_df.drop(columns=['LowQualityArea', 'SeasonalPorchArea', 'ScreenPorchArea', 'SwimmingPoolArea', 'ExtraFacilityValue'])
print(train_df.shape)
print(test_df.shape)

(1162, 70)
(260, 69)


In [641]:
# import seaborn as sns
# discrete_numeric_cols.remove('SwimmingPoolArea')
# n_features = len(discrete_numeric_cols)
# n_cols = 2
# n_rows = math.ceil(n_features / n_cols)

# # Create the figure and axes
# fig, axes = plt.subplots(n_rows, n_cols, figsize=(8 * n_cols, 6 * n_rows))
# axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

# # Loop through the identified columns and create a boxplot for each
# for idx, feature in enumerate(discrete_numeric_cols):
#     sns.boxplot(x=feature, y='HotelValue', data=train_df, ax=axes[idx])
#     axes[idx].set_title(f'HotelValue vs {feature}', fontsize=14)
#     axes[idx].set_xlabel(feature, fontsize=12)
#     axes[idx].set_ylabel('HotelValue', fontsize=12)

# # Hide any unused subplots
# for i in range(n_features, len(axes)):
#     fig.delaxes(axes[i])

# # Adjust layout to prevent overlap and display the plots
# plt.tight_layout()
# plt.show()

In [642]:
#We will drop features which show very little variation across categories as these are unlikely to be string predictors of the target variable.
train_df = train_df.drop(columns=['MonthSold', 'BasementHalfBaths', 'YearSold'])
test_df = test_df.drop(columns=['MonthSold', 'BasementHalfBaths', 'YearSold'])

print(train_df.shape)
print(test_df.shape)

(1162, 67)
(260, 66)


In [643]:
categorical_missing = missing_summary[
    (missing_summary["missing_percent"] > 0) &
    (missing_summary["dtype"]=="object")    
]

print(categorical_missing)


missing_counts = train_df.isnull().sum()
print(missing_counts[missing_counts > 0]) 

missing_counts_test = test_df.isnull().sum()
print(missing_counts_test[missing_counts_test > 0]) 

                  missing_percent   dtype
ParkingFinish            4.991394  object
ParkingType              4.991394  object
ParkingQuality           4.991394  object
ParkingCondition         4.991394  object
ParkingType         58
ParkingFinish       58
ParkingQuality      58
ParkingCondition    58
dtype: int64
BasementHeight            8
BasementCondition         8
BasementExposure          8
BasementFacilityType1     8
BasementFacilityType2     9
ParkingType              16
ParkingFinish            16
ParkingQuality           16
ParkingCondition         16
dtype: int64


In [644]:
# categorical_cols = train_df.select_dtypes(include=['object']).columns

# print(f"Found {len(categorical_cols)} columns with 'object' data type:")
# target_variable = 'HotelValue'

# # Determine the grid size for the subplots (arranging them in 2 columns)
# n_features = len(categorical_cols)
# n_cols = 2
# n_rows = math.ceil(n_features / n_cols)

# # Create the figure and axes
# fig, axes = plt.subplots(n_rows, n_cols, figsize=(8 * n_cols, 6 * n_rows))
# axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

# # Loop through the identified columns and create a boxplot for each
# for idx, feature in enumerate(categorical_cols):
#     # It's good practice to sort the categories for a cleaner plot, e.g., by median value
#     # This is optional but makes the plot easier to interpret
#     sorted_order = train_df.groupby(feature)[target_variable].median().sort_values().index
    
#     sns.boxplot(x=feature, y=target_variable, data=train_df, ax=axes[idx], order=sorted_order)
    
#     axes[idx].set_title(f'{target_variable} vs {feature}', fontsize=14)
#     axes[idx].set_xlabel(feature, fontsize=12)
#     axes[idx].set_ylabel(target_variable, fontsize=12)
    
#     # Optional: Rotate x-axis labels if they are long and overlap
#     axes[idx].tick_params(axis='x', rotation=45)

# # Hide any unused subplots
# for i in range(n_features, len(axes)):
#     fig.delaxes(axes[i])

# # Adjust layout to prevent overlap and display the plots
# plt.tight_layout()
# plt.show()

In [645]:
#We will drop columns with very little variance across categories
train_df = train_df.drop(columns=['UtilityAccess', 'RoadType'])
test_df = test_df.drop(columns=['UtilityAccess', 'RoadType'])

print(train_df.shape)
print(test_df.shape)

(1162, 65)
(260, 64)


In [646]:
numeric_cols = train_df.select_dtypes(include=[np.number]).columns
negative_values = (train_df[numeric_cols] < 0).sum()
print("Negative values per column:\n", negative_values[negative_values > 0])

# Check for infinite values
infinite_values = np.isinf(train_df[numeric_cols]).sum()
print("Infinite values per column:\n", infinite_values[infinite_values > 0])

# # Visualize
# plt.figure(figsize=(20, 16))
# sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
# plt.title('Correlation Matrix')
# plt.show()

# # Find high correlations (excluding diagonal)
# high_corr = []
# for i in range(len(corr_matrix.columns)):
#     for j in range(i+1, len(corr_matrix.columns)):
#         if abs(corr_matrix.iloc[i, j]) > 0.75:  # threshold
#             high_corr.append({
#                 'Feature1': corr_matrix.columns[i],
#                 'Feature2': corr_matrix.columns[j],
#                 'Correlation': corr_matrix.iloc[i, j]
#             })

# print("Highly correlated pairs (>0.75):")
# for pair in high_corr:
#     print(f"{pair['Feature1']} <-> {pair['Feature2']}: {pair['Correlation']:.3f}")

train_df=train_df.drop(columns=['ParkingConstructionYear'])
test_df=test_df.drop(columns=['ParkingConstructionYear'])
print(train_df.shape)
print(test_df.shape)

Negative values per column:
 Series([], dtype: int64)
Infinite values per column:
 Series([], dtype: int64)
(1162, 64)
(260, 63)


In [648]:
X = train_encoded.drop(columns=['HotelValue','Id'])
test_ids = test_encoded['Id'].copy()
y = train_encoded['HotelValue']
X_test = test_encoded.drop(columns=['Id'])
print(X.shape)
print(y.shape)
print(X_test.shape)


null_value_percentages=(test_encoded.isna().sum()/test_encoded.shape[0])*100
null_value_percentages=null_value_percentages[null_value_percentages>0]
null_value_percentages

(1162, 156)
(1162,)
(260, 156)


Series([], dtype: float64)

In [649]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Prepare data
X = train_encoded.drop(['HotelValue', 'Id'], axis=1)
y = train_encoded['HotelValue']
X_test = test_encoded.drop(columns=['Id'])

# Apply log transformation to target
y = np.log1p(y)

# Handle categorical variables - FIX for unseen categories
categorical_columns = X.select_dtypes(include=['object']).columns

# Combine train and test to fit encoder on all categories
for col in categorical_columns:
    le = LabelEncoder()
    
    # Fit on combined data to see all categories
    combined = pd.concat([X[col], X_test[col]], axis=0).astype(str)
    le.fit(combined)
    
    # Transform both datasets
    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

print(f"Categorical encoding complete with {len(categorical_columns)} columns")

# Train linear regression (without scaling)
print("\n=== Training Linear Regression ===")
lr_model = LinearRegression()
lr_model.fit(X, y)


# Make test predictions
y_test_pred = lr_model.predict(X_test)
y_test_pred_actual = np.expm1(y_test_pred)

print(f"\nTest Predictions:")
print(f"Sample: {y_test_pred_actual[:5]}")
print(f"Min: ${y_test_pred_actual.min():.2f}")
print(f"Max: ${y_test_pred_actual.max():.2f}")
print(f"Mean: ${y_test_pred_actual.mean():.2f}")

# Create submission
submission = pd.DataFrame({
    'Id': test_ids,
    'HotelValue': y_test_pred_actual
})

submission.to_csv('submission_linear_regression.csv', index=False)
print("\n Submission saved as 'submission.csv'")
print(submission.head(10))

Categorical encoding complete with 0 columns

=== Training Linear Regression ===

Test Predictions:
Sample: [ 9561.9663817  23117.33726856  7172.03825535 10587.95190572
 20741.43211   ]
Min: $2915.19
Max: $725360.27
Mean: $17551.12

 Submission saved as 'submission.csv'
     Id    HotelValue
0   893   9561.966382
1  1106  23117.337269
2   414   7172.038255
3   523  10587.951906
4  1037  20741.432110
5   615   5879.415835
6   219  14995.095928
7  1161   8829.874775
8   650   5518.315622
9   888   8809.158220


In [ ]:
# Ordinal Encoding for quality-related columns
ordinal_cols = ['ExteriorQuality', 'ExteriorCondition', 'HeatingQuality', 'KitchenQuality']

# Define ordinal mappings
ordinal_mappings = {
    'Ex': 5,  # Excellent
    'Gd': 4,  # Good
    'TA': 3,  # Typical/Average
    'Av': 3,  # Some datasets use 'Av' instead of 'TA'
    'Fa': 2,  # Fair
    'Mn': 2,  # Sometimes used instead of 'Fa'
    'Po': 1,  # Poor
    'No': 0,  # None
    'None': 0
}

def encode_ordinal(train_df, test_df, ordinal_cols, mapping):
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()

    for col in ordinal_cols:
        if col in train_encoded.columns:
            train_encoded[col] = train_encoded[col].map(mapping).fillna(0).astype(int)
        if col in test_encoded.columns:
            test_encoded[col] = test_encoded[col].map(mapping).fillna(0).astype(int)

    return train_encoded, test_encoded

# Apply encoding
train_df, test_df = encode_ordinal(train_df, test_df, ordinal_cols, ordinal_mappings)
